## Свёрточные нейронные сети и аугментация данных
В этом задании необходимо обучить конволюционную нейросеть типа сети LeNet5, предложенную Яном Лекуном в 1998 году, для распознавания рукописных цифр из базы данных MNIST.  
Код ниже предлагает базовый вариант обучения, который даёт около 98-99% точности на тестовой выборке.  

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import  v2
from matplotlib import pyplot  as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

cpu


# 1 Этап - подготовка данных!

In [2]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# 2 Этап - Нейронная сеть!

<img src='Lenet5.png'>

In [3]:
# Fully connected neural network with one hidden layer
class Lenet5(nn.Module):
    def __init__(self, hidden_size1=500, num_classes=10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=0)
        self.batchnorm1 = nn.BatchNorm2d(20)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1, padding=0)
        self.batchnorm2 = nn.BatchNorm2d(50)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(50 * 4 * 4, hidden_size1) # fully connected
        self.fc2 = nn.Linear(hidden_size1, num_classes)  
    
    def forward(self, x): # вход размера 1 x 28 x 28
        out = self.conv1(x) # на выходе размер 20 x 24 x 24
        out = self.batchnorm1(out)
        out = self.maxpool(out) # на выходе размер 20 x 12 x 12
        out = self.relu(out)
        out = self.conv2(out)  # на выходе размер 50 x 8 x 8
        out = self.batchnorm2(out)
        out = self.maxpool(out) # на выходе размер 50 x 4 x 4
        out = self.relu(out)
        out = out.reshape(-1, 50 * 4 * 4) # на выходе размер 800
        out = self.fc1(out) # на выходе размер 500
        out = self.relu(out)
        out = self.fc2(out) # на выходе размер 10
        
        return out

model = Lenet5(hidden_size1=500, num_classes=10).to(device)


# 3 этап - Loss-функция и оптимизатор

In [4]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 4 этап - обучение нейросети

In [5]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.0528
Epoch [1/5], Step [200/600], Loss: 0.1026
Epoch [1/5], Step [300/600], Loss: 0.0721
Epoch [1/5], Step [400/600], Loss: 0.0329
Epoch [1/5], Step [500/600], Loss: 0.0157
Epoch [1/5], Step [600/600], Loss: 0.0287
Accuracy of the network on the 10000 test images: 98.5 %
Epoch [2/5], Step [100/600], Loss: 0.0257
Epoch [2/5], Step [200/600], Loss: 0.0190
Epoch [2/5], Step [300/600], Loss: 0.1821
Epoch [2/5], Step [400/600], Loss: 0.1024
Epoch [2/5], Step [500/600], Loss: 0.0642
Epoch [2/5], Step [600/600], Loss: 0.0917
Accuracy of the network on the 10000 test images: 98.47 %
Epoch [3/5], Step [100/600], Loss: 0.0169
Epoch [3/5], Step [200/600], Loss: 0.0340
Epoch [3/5], Step [300/600], Loss: 0.0084
Epoch [3/5], Step [400/600], Loss: 0.0372
Epoch [3/5], Step [500/600], Loss: 0.0595
Epoch [3/5], Step [600/600], Loss: 0.0137
Accuracy of the network on the 10000 test images: 99.05 %
Epoch [4/5], Step [100/600], Loss: 0.0545
Epoch [4/5], Step [200/600], 

# Улучшение кода: аугментация данных
Расширим обучающий датасет за счёт дополнительных искажений исходных картинок:  
 - небольших случайных поворотов (rotate), 
 - дрожания (jitter), 
 - кроппинга (crop) 
 - и др.   
Для трансформации картинок используем методы библиотеки `torchvision.transforms`, про которые можно прочитать [здесь](https://pytorch.org/vision/stable/transforms.html)  
Целью задания является - получить точность на тестовой выборке 99.5%.

## Подготовка данных

In [9]:
transformеr_for_train = v2.Compose([transforms.ToTensor(),
                                    v2.CenterCrop(28),
                                    v2.RandomRotation(degrees=15),
                                    v2.Normalize(mean=[0.456], std=[0.224])])

transformеr_for_test = v2.Compose([transforms.ToTensor(),
                                   v2.Normalize(mean=[0.456], std=[0.224])])

In [10]:
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transformеr_for_train,  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transformеr_for_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Обучение нейросети

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    model.train() # включены dropout (зануление некоторых нейронов) и batchnormalization
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        #images = images.reshape(-1, 28*28).to(device)
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    # Test the model
    # In test phase, we don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        model.eval() # отключены dropout (зануление некоторых нейронов) и batchnormalization
        correct = 0
        total = 0
        for images, labels in test_loader:
            #images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

    # Save the model checkpoint
    torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/5], Step [100/600], Loss: 0.0022
Epoch [1/5], Step [200/600], Loss: 0.0015
Epoch [1/5], Step [300/600], Loss: 0.0023
Epoch [1/5], Step [400/600], Loss: 0.0006
Epoch [1/5], Step [500/600], Loss: 0.0174
Epoch [1/5], Step [600/600], Loss: 0.0026
Accuracy of the network on the 10000 test images: 99.27 %
Epoch [2/5], Step [100/600], Loss: 0.0008
Epoch [2/5], Step [200/600], Loss: 0.0173
Epoch [2/5], Step [300/600], Loss: 0.0934
Epoch [2/5], Step [400/600], Loss: 0.0427
Epoch [2/5], Step [500/600], Loss: 0.0052
Epoch [2/5], Step [600/600], Loss: 0.0061
Accuracy of the network on the 10000 test images: 99.34 %
Epoch [3/5], Step [100/600], Loss: 0.0209
Epoch [3/5], Step [200/600], Loss: 0.0001
Epoch [3/5], Step [300/600], Loss: 0.0184
Epoch [3/5], Step [400/600], Loss: 0.0352
Epoch [3/5], Step [500/600], Loss: 0.0044
Epoch [3/5], Step [600/600], Loss: 0.0107
Accuracy of the network on the 10000 test images: 99.29 %
Epoch [4/5], Step [100/600], Loss: 0.0020
Epoch [4/5], Step [200/600],